In [2]:
import pandas as pd 
import csv
import mysql.connector

In [10]:
# connecting to mySQL
mydb = mysql.connector.connect(
    host = "localhost",
    database = "mydatabase",
    user = "root",
    password = "LasVegas1!"
)

print(mydb)

In [47]:
# creating the database
#mycursor = mydb.cursor()
#mycursor.execute("CREATE DATABASE mydatabase")

In [48]:
# making sure db exists
cursor = mydb.cursor()
cursor.execute("SHOW DATABASES")
for x in cursor:
  print(x)
cursor.close()

('information_schema',)
('mydatabase',)
('mysql',)
('performance_schema',)
('sys',)


True

In [5]:
# importing monthly flight data from BTS
mar20 = pd.read_csv('03_2020.csv')
apr20 = pd.read_csv('04_2020.csv')
may20 = pd.read_csv('05_2020.csv')
jun20 = pd.read_csv('06_2020.csv')
jul20 = pd.read_csv('07_2020.csv')
aug20 = pd.read_csv('08_2020.csv')
sep20 = pd.read_csv('09_2020.csv')
oct20 = pd.read_csv('10_2020.csv')
nov20 = pd.read_csv('11_2020.csv')
dec20 = pd.read_csv('12_2020.csv')
jan21 = pd.read_csv('01_2021.csv')
feb21 = pd.read_csv('02_2021.csv')
mar21 = pd.read_csv('03_2021.csv')
apr21 = pd.read_csv('04_2021.csv')
may21 = pd.read_csv('05_2021.csv')
jun21 = pd.read_csv('06_2021.csv')
jul21 = pd.read_csv('07_2021.csv')

flightsdfs = [mar20, apr20, may20, jun20, 
              jul20, aug20, sep20, oct20, 
              nov20, dec20, jan21, feb21, 
              mar21, apr21, may21, jun21, jul21]

In [6]:
# making a single df 
flightsdata = pd.concat([mar20, apr20, may20, jun20, 
              jul20, aug20, sep20, oct20, 
              nov20, dec20, jan21, feb21, 
              mar21, apr21, may21, jun21, jul21], join = "inner")

flightsdata = flightsdata.drop(flightsdata.columns[[16]], axis = 1)
flightsdata = flightsdata.where((pd.notnull(flightsdata)), None)

In [7]:
flightsdata.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID,DEST_STATE_ABR,DEP_TIME,CANCELLED,CANCELLATION_CODE
0,2020,3,16,1,2020-03-16,AA,8,12173,32134,HI,11298,30194,TX,1757.0,0,None
1,2020,3,17,2,2020-03-17,AA,8,12173,32134,HI,11298,30194,TX,1800.0,0,None
2,2020,3,18,3,2020-03-18,AA,8,12173,32134,HI,11298,30194,TX,1759.0,0,None
3,2020,3,19,4,2020-03-19,AA,8,12173,32134,HI,11298,30194,TX,1759.0,0,None
4,2020,3,20,5,2020-03-20,AA,8,12173,32134,HI,11298,30194,TX,1833.0,0,None


In [8]:
flightsdata.to_csv("allflightsdata.csv", index = False, header = False)

In [11]:
if mydb.is_connected():
    cursor = mydb.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    
    # create table and insert 16 variables 
    cursor.execute('DROP TABLE IF EXISTS flights;')
    print('Creating table....')
    
    cursor.execute("CREATE TABLE flights(year varchar(10), month varchar(10), day_of_month varchar(10), day_of_week varchar(10), fl_date varchar(10), op_unique_carrier varchar(10), op_carrier_fl_num varchar(10), origin_airport_id varchar(10), origin_city_id varchar(10), origin_state_abr varchar(10), dest_airport_id varchar(10), dest_city_id varchar(10), dest_state_abr varchar(10), dep_time varchar(10), cancelled varchar(10), cancellation_code varchar(10))")
    print("flights table is created....") 

    # input csv files into table 
    for i, row in flightsdata.iterrows():
        sql = "INSERT INTO mydatabase.flights VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, tuple(row))
       # print("record inserted")
        mydb.commit()
    cursor.close()
    print("data inserted")


Creating table....
flights table is created....
data inserted


In [13]:
print(len(flightsdata))

7118895


In [49]:
cursor = mydb.cursor()
cursor.execute("SHOW TABLES")

for x in cursor:
  print(x)
cursor.close()

('cancelledflights',)
('flights',)


True

In [57]:
cursor.reset()

In [94]:
cursor = mydb.cursor()
cursor.execute("DROP TABLE IF EXISTS cancelledflights;")

In [96]:
# table of only flights that were cancelled 
cursor = mydb.cursor()
cursor.execute('''DROP TABLE IF EXISTS cancelledflights''')
cursor.execute('''CREATE TABLE cancelledflights LIKE flights''')
cursor.execute('''INSERT INTO cancelledflights 
SELECT * FROM flights WHERE cancelled = 1''')
cursor.close()

True

In [99]:
cursor = mydb.cursor()
sql = "SELECT COUNT(*) FROM cancelledflights"
cursor.execute(sql)
for x in cursor:
  print(x)
cursor.close()

(341106,)


True

In [100]:
cursor = mydb.cursor()
cursor.execute("SELECT COUNT(*) FROM flights")
for x in cursor:
  print(x)
cursor.close()

(7118895,)


True

In [68]:
cursor = mydb.cursor()
sql = "SELECT COUNT(*) FROM grouped_state"
cursor.execute(sql)
for x in cursor:
  print(x)

(27004,)


In [102]:
flightsdata.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST_CITY_MARKET_ID,DEST_STATE_ABR,DEP_TIME,CANCELLED,CANCELLATION_CODE
0,2020,3,16,1,2020-03-16,AA,8,12173,32134,HI,11298,30194,TX,1757.0,0,None
1,2020,3,17,2,2020-03-17,AA,8,12173,32134,HI,11298,30194,TX,1800.0,0,None
2,2020,3,18,3,2020-03-18,AA,8,12173,32134,HI,11298,30194,TX,1759.0,0,None
3,2020,3,19,4,2020-03-19,AA,8,12173,32134,HI,11298,30194,TX,1759.0,0,None
4,2020,3,20,5,2020-03-20,AA,8,12173,32134,HI,11298,30194,TX,1833.0,0,None


In [74]:
cursor = mydb.cursor()
cursor.execute("DROP TABLE IF EXISTS grouped_state")
cursor.execute("CREATE TABLE grouped_state LIKE flights")
cursor.execute("INSERT INTO grouped_state SELECT * FROM flights")
cursor.execute("SELECT dest_state_abr, fl_date, COUNT(*) FROM grouped_state GROUP BY dest_state_abr, fl_date")

In [61]:
cursor = mydb.cursor()
cursor.execute("DROP TABLE IF EXISTS grouped_state")
cursor.execute("CREATE TABLE grouped_state LIKE flights")
cursor.execute("INSERT INTO grouped_state SELECT * FROM flights WHERE cancelled != 1")
cursor.execute("SELECT dest_state_abr, fl_date, COUNT(*) FROM grouped_state GROUP BY dest_state_abr, fl_date")

In [105]:
cursor = mydb.cursor()
cursor.execute("DROP TABLE IF EXISTS grouped_state")
cursor.execute('''CREATE TABLE grouped_state AS 
SELECT dest_state_abr, fl_date, COUNT(*) FROM flights WHERE cancelled != 1
GROUP BY dest_state_abr, fl_date
''')
cursor.close()

True

In [106]:
cursor = mydb.cursor()
cursor.execute("DROP TABLE IF EXISTS grouped_state_origin")
cursor.execute('''CREATE TABLE grouped_state_origin AS 
SELECT dest_state_abr, origin_state_abr, fl_date, COUNT(*) FROM flights WHERE cancelled != 1
GROUP BY dest_state_abr, origin_state_abr, fl_date
''')
cursor.close()

True

In [12]:
from sqlalchemy import create_engine
import mysql.connector as connection 

In [81]:
mydbconn = connection.connect(host = "localhost", database = "mydatabase", user = "root", password = "LasVegas1!")
sql = ('SELECT * FROM grouped_state;')
grouped_state = pd.read_sql(sql, mydbconn)
mydbconn.close()

grouped_state.head()

In [109]:
mydbconn = connection.connect(host = "localhost", database = "mydatabase", user = "root", password = "LasVegas1!")
sql = ('SELECT * FROM grouped_state_origin;')
grouped_state_origin = pd.read_sql(sql, mydbconn)
mydbconn.close()

In [13]:
mydbconn = connection.connect(host = "localhost", database = "mydatabase", user = "root", password = "LasVegas1!")
sql = ('SELECT dest_state_abr, origin_state_abr, fl_date FROM flights;')
allflightsdata = pd.read_sql(sql, mydbconn)
mydbconn.close()

In [110]:
grouped_state_origin.head()

,dest_state_abr,origin_state_abr,fl_date,COUNT(*)
0,TX,HI,2020-03-16,6
1,TX,HI,2020-03-17,5
2,TX,HI,2020-03-18,6
3,TX,HI,2020-03-19,6
4,TX,HI,2020-03-20,6


In [111]:
grouped_state_origin.to_csv("grouped_state_origin.csv", index = False, header = False)

In [90]:
grouped_state.to_csv("grouped_state3.csv", index = False, header = False)

In [14]:
allflightsdata.to_csv("allflightsdata2.csv")